In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

In [3]:
Expressions={0:"Angry",1:"Disgust",2:"Fear",3:"Happy",4:"Neutral",5:"Sad",6:"Surprise"}

In [4]:
from tensorflow.keras.backend import sigmoid

In [5]:
def swish(x, beta = 1):
  return (x * sigmoid(beta * x))

In [6]:
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation

In [7]:
get_custom_objects().update({'swish': Activation(swish)})

In [8]:
model_fer=tf.keras.models.load_model("C:\\Users\\pushkar\\Desktop\\fer2013\\fer2013_finalmodels\\model_fer_swish.h5")
model_kdef=tf.keras.models.load_model("C:\\Users\\pushkar\\Desktop\\kdef\\kdef_final_models\\model_simple_kd_bw_modified1(Batchnorm).h5")

In [9]:
def detect_face(image):
    cascPath = 'haarcascade_frontalface_default.xml'
    faceCascade = cv2.CascadeClassifier(cascPath)
  #BGR to gray conversion
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #Cascade multiscale classifier
    detected_faces = faceCascade.detectMultiScale(image, minNeighbors=6,
                                                minSize=(100,100))
    sub = gray

    for x, y, w, h in detected_faces:
        sub = gray[y:y+h,x:x+w]
    return sub,detected_faces

In [10]:
def make_prediction(frame):
    frame1=cv2.resize(frame,(200,200),interpolation = cv2.INTER_AREA)
    frame2=cv2.resize(frame,(48,48),interpolation = cv2.INTER_AREA)
    frame1=frame1/255.0
    frame1=np.array(frame1).reshape(1,200,200,1)
    
    frame2=frame2/255.0     
    frame2=np.array(frame2).reshape(1,48,48,1)
    
    predict2=model_fer.predict(frame2)
    predict1=model_kdef.predict(frame1)
    print('kdef',predict1)
    print('fer wrong',predict2)
    sad_val=predict2[0][4]
    surp_val=predict2[0][5]
    ne_val =predict2[0][6]

    predict2[0][4],predict2[0][5],predict2[0][6]=ne_val,sad_val,surp_val
    print('fer right',predict2)
    predict=np.add(predict1,predict2)
    predict=predict/2
    print('overall predict',predict)
    predict_max=np.argmax(predict)
    return predict,predict_max

In [15]:
def face_in_video():
    cap=cv2.VideoCapture("mixed_emotions7.mp4")
    intensity_label=""
    while True:
        ret,img=cap.read()
        if ret == False:
            break
        print('image',img.shape)
        x=img.shape[0]
        y=img.shape[1]
        y1=int(y/(x/300))
        print(y1)
        img2 = cv2.resize(img,(y1,300),interpolation = cv2.INTER_AREA)
        sub_faceog,detected_faces_og=detect_face(img)
        sub_face,detected_faces = detect_face(img2)
        print('face shape :',sub_face.shape)
        res_array,res=make_prediction(sub_face)
        res_array=np.squeeze(res_array)
        intensity_value=res_array[res]
        if(intensity_value>=0.65):
            intensity_label="Intensity: Extreme"
        elif(intensity_value>=0.4 and intensity_value<0.65):
            intensity_label="Intensity: Moderate"
        elif(intensity_value<0.4):
            intensity_label="Intensity: Low"
        
        for x,y,w,h in detected_faces_og:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img,str(Expressions[res])+" "+intensity_label,(x,y-5),font,0.5,(205,200,50),1,cv2.LINE_AA)
            cv2.putText(img,"Anger: "+str(round(res_array[0]*100,2))+"%",(5,20),font,0.5,(205,200,0),1,cv2.LINE_AA)
            cv2.rectangle(img,(150,10),(150+int(round(res_array[0]*200)),20),(255,0,0),thickness=cv2.FILLED)
            
            cv2.putText(img,"Disgust: "+str(round(res_array[1]*100,2))+"%",(5,50),font,0.5,(205,200,0),1,cv2.LINE_AA)
            cv2.rectangle(img,(150,40),(150+int(round(res_array[1]*200)),50),(255,0,0),thickness=cv2.FILLED)
            
            cv2.putText(img,"Fear: "+str(round(res_array[2]*100,2))+"%",(5,80),font,0.5,(205,200,0),1,cv2.LINE_AA)
            cv2.rectangle(img,(150,70),(150+int(round(res_array[2]*200)),80),(255,0,0),thickness=cv2.FILLED)
            
            cv2.putText(img,"Happy: "+str(round(res_array[3]*100,2))+"%",(5,110),font,0.5,(205,200,0),1,cv2.LINE_AA)
            cv2.rectangle(img,(150,100),(150+int(round(res_array[3]*200)),110),(255,0,0),thickness=cv2.FILLED)
            
            cv2.putText(img,"Neutral: "+str(round(res_array[4]*100,2))+"%",(5,140),font,0.5,(205,200,0),1,cv2.LINE_AA)
            cv2.rectangle(img,(150,130),(150+int(round(res_array[4]*200)),140),(255,0,0),thickness=cv2.FILLED)
            
            cv2.putText(img,"Sad: "+str(round(res_array[5]*100,2))+"%",(5,170),font,0.5,(205,200,0),1,cv2.LINE_AA)
            cv2.rectangle(img,(150,160),(150+int(round(res_array[5]*200)),170),(255,0,0),thickness=cv2.FILLED)
            
            cv2.putText(img,"Surprise: "+str(round(res_array[6]*100,2))+"%",(5,200),font,0.5,(205,200,0),1,cv2.LINE_AA)
            cv2.rectangle(img,(150,190),(150+int(round(res_array[6]*200)),200),(255,0,0),thickness=cv2.FILLED)
        cv2.imshow('img',img)
        if cv2.waitKey(1) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [14]:
face_in_video()

image (720, 1280, 3)
533
face shape : (132, 132)
kdef [[1.5521441e-01 4.0746272e-01 3.4535443e-03 3.2921080e-04 1.3058114e-01
  2.9846126e-01 4.4977334e-03]]
fer wrong [[2.8072180e-02 1.2275066e-04 1.5174714e-02 1.3236044e-02 4.1985024e-02
  4.9062534e-03 8.9650309e-01]]
fer right [[2.8072180e-02 1.2275066e-04 1.5174714e-02 1.3236044e-02 8.9650309e-01
  4.1985024e-02 4.9062534e-03]]
overall predict [[0.0916433  0.20379274 0.00931413 0.00678263 0.5135421  0.17022315
  0.00470199]]
image (720, 1280, 3)
533
face shape : (131, 131)
kdef [[0.17314292 0.34287238 0.0046012  0.00048149 0.16091429 0.3144161
  0.00357165]]
fer wrong [[2.43912488e-02 1.03289356e-04 1.44586395e-02 1.37297474e-02
  3.98182757e-02 4.60599083e-03 9.02892768e-01]]
fer right [[2.43912488e-02 1.03289356e-04 1.44586395e-02 1.37297474e-02
  9.02892768e-01 3.98182757e-02 4.60599083e-03]]
overall predict [[0.09876709 0.17148784 0.00952992 0.00710562 0.5319035  0.1771172
  0.00408882]]
image (720, 1280, 3)
533
face shape : (

face shape : (132, 132)
kdef [[0.36930943 0.31244847 0.00304654 0.00044466 0.10052759 0.20947358
  0.00474969]]
fer wrong [[3.1599607e-02 1.2577434e-04 1.5861893e-02 1.0853487e-02 4.8493735e-02
  3.9415294e-03 8.8912398e-01]]
fer right [[3.1599607e-02 1.2577434e-04 1.5861893e-02 1.0853487e-02 8.8912398e-01
  4.8493735e-02 3.9415294e-03]]
overall predict [[0.20045452 0.15628712 0.00945422 0.00564907 0.49482578 0.12898366
  0.00434561]]
image (720, 1280, 3)
533
face shape : (133, 133)
kdef [[0.31313002 0.3578595  0.00421478 0.00045556 0.08697087 0.23133
  0.0060392 ]]
fer wrong [[2.8994991e-02 1.2536158e-04 1.5096861e-02 1.2549878e-02 4.3684028e-02
  4.5685391e-03 8.9498037e-01]]
fer right [[2.8994991e-02 1.2536158e-04 1.5096861e-02 1.2549878e-02 8.9498037e-01
  4.3684028e-02 4.5685391e-03]]
overall predict [[0.1710625  0.17899242 0.00965582 0.00650272 0.49097562 0.13750702
  0.00530387]]
image (720, 1280, 3)
533
face shape : (134, 134)
kdef [[0.3903462  0.09788132 0.00342335 0.00049964 

face shape : (127, 127)
kdef [[9.9120152e-01 6.1715515e-03 4.8752758e-05 7.8924577e-06 1.0521420e-04
  2.4609331e-03 4.1432490e-06]]
fer wrong [[0.7794188  0.00892713 0.03500484 0.01103104 0.0581643  0.00700456
  0.10044916]]
fer right [[0.7794188  0.00892713 0.03500484 0.01103104 0.10044916 0.0581643
  0.00700456]]
overall predict [[0.8853102  0.00754934 0.0175268  0.00551947 0.05027719 0.03031262
  0.00350435]]
image (720, 1280, 3)
533
face shape : (127, 127)
kdef [[9.86141980e-01 1.05240699e-02 5.65078735e-05 1.12360185e-05
  1.40805976e-04 3.11824121e-03 7.00123701e-06]]
fer wrong [[0.8303973  0.0091066  0.033243   0.00892073 0.04883472 0.00547744
  0.06402011]]
fer right [[0.8303973  0.0091066  0.033243   0.00892073 0.06402011 0.04883472
  0.00547744]]
overall predict [[0.90826964 0.00981533 0.01664976 0.00446598 0.03208046 0.02597648
  0.00274222]]
image (720, 1280, 3)
533
face shape : (127, 127)
kdef [[9.8597890e-01 1.0714168e-02 5.5659795e-05 1.1115463e-05 1.3837911e-04
  3.094

face shape : (128, 128)
kdef [[9.9491030e-01 4.4900691e-03 6.5008731e-05 4.4614767e-06 5.8950074e-05
  4.6309183e-04 8.1068147e-06]]
fer wrong [[0.84875935 0.02891945 0.0550369  0.00530561 0.04019098 0.00590827
  0.01587942]]
fer right [[0.84875935 0.02891945 0.0550369  0.00530561 0.01587942 0.04019098
  0.00590827]]
overall predict [[0.9218348  0.01670476 0.02755095 0.00265504 0.00796919 0.02032704
  0.00295819]]
image (720, 1280, 3)
533
face shape : (128, 128)
kdef [[9.9472636e-01 4.6107536e-03 6.4964617e-05 4.8944212e-06 5.9695441e-05
  5.2476814e-04 8.6631962e-06]]
fer wrong [[0.8682187  0.02351012 0.04923647 0.00472645 0.03593483 0.00477454
  0.01359886]]
fer right [[0.8682187  0.02351012 0.04923647 0.00472645 0.01359886 0.03593483
  0.00477454]]
overall predict [[0.93147254 0.01406044 0.02465072 0.00236567 0.00682928 0.0182298
  0.0023916 ]]
image (720, 1280, 3)
533
face shape : (130, 130)
kdef [[9.9057651e-01 7.7761542e-03 1.1083111e-04 5.7038260e-06 1.5413621e-04
  1.3376762e-0

face shape : (129, 129)
kdef [[9.9216497e-01 6.9695334e-03 9.2366754e-05 8.3775203e-06 1.1998705e-04
  6.2764424e-04 1.7141707e-05]]
fer wrong [[0.9070728  0.00762481 0.0252465  0.00490547 0.03191414 0.00231891
  0.02091736]]
fer right [[0.9070728  0.00762481 0.0252465  0.00490547 0.02091736 0.03191414
  0.00231891]]
overall predict [[0.9496189  0.00729717 0.01266943 0.00245692 0.01051867 0.01627089
  0.00116802]]
image (720, 1280, 3)
533
face shape : (128, 128)
kdef [[9.6653664e-01 2.9965710e-02 2.4824616e-04 1.9981935e-05 5.2771281e-04
  2.6219634e-03 7.9778401e-05]]
fer wrong [[0.94239134 0.00567545 0.01829656 0.00257119 0.02102003 0.00105133
  0.00899415]]
fer right [[0.94239134 0.00567545 0.01829656 0.00257119 0.00899415 0.02102003
  0.00105133]]
overall predict [[9.5446396e-01 1.7820578e-02 9.2724022e-03 1.2955873e-03 4.7609326e-03
  1.1820999e-02 5.6555291e-04]]
image (720, 1280, 3)
533
face shape : (130, 130)
kdef [[9.8608541e-01 1.2753045e-02 2.0243194e-04 7.1282202e-06 2.6319

...

face shape : (127, 127)
kdef [[2.7164424e-04 2.1939059e-03 1.3396904e-01 1.2071067e-05 1.3545340e-04
  2.5845176e-05 8.6339200e-01]]
fer wrong [[2.0193437e-03 9.9617673e-06 5.8545422e-02 4.6607810e-03 6.1729946e-04
  9.3398178e-01 1.6529908e-04]]
fer right [[2.0193437e-03 9.9617673e-06 5.8545422e-02 4.6607810e-03 1.6529908e-04
  6.1729946e-04 9.3398178e-01]]
overall predict [[1.1454939e-03 1.1019338e-03 9.6257232e-02 2.3364259e-03 1.5037623e-04
  3.2157233e-04 8.9868689e-01]]
image (720, 1280, 3)
533
face shape : (127, 127)
kdef [[4.0489802e-04 2.9462564e-03 1.5180685e-01 1.5311776e-05 1.9558947e-04
  4.5398589e-05 8.4458572e-01]]
fer wrong [[2.0517628e-03 1.0313088e-05 5.8563326e-02 4.7258181e-03 6.3243322e-04
  9.3384361e-01 1.7279254e-04]]
fer right [[2.0517628e-03 1.0313088e-05 5.8563326e-02 4.7258181e-03 1.7279254e-04
  6.3243322e-04 9.3384361e-01]]
overall predict [[1.22833042e-03 1.47828471e-03 1.05185084e-01 2.37056497e-03
  1.84191013e-04 3.38915910e-04 8.89214635e-01]]
image 

face shape : (127, 127)
kdef [[0.21875525 0.0875442  0.24277683 0.00063299 0.01013696 0.00090532
  0.4392485 ]]
fer wrong [[0.01750212 0.00094991 0.03710554 0.02656109 0.01383431 0.46879405
  0.43525296]]
fer right [[0.01750212 0.00094991 0.03710554 0.02656109 0.43525296 0.01383431
  0.46879405]]
overall predict [[0.11812868 0.04424705 0.13994119 0.01359704 0.22269496 0.00736981
  0.45402128]]
image (720, 1280, 3)
533
face shape : (126, 126)
kdef [[0.3195786  0.06109346 0.40075934 0.00062662 0.01076924 0.00054583
  0.20662692]]
fer wrong [[0.01925167 0.00095856 0.03312237 0.02966183 0.01634078 0.30264014
  0.59802467]]
fer right [[0.01925167 0.00095856 0.03312237 0.02966183 0.59802467 0.01634078
  0.30264014]]
overall predict [[0.16941513 0.03102601 0.21694085 0.01514422 0.30439696 0.0084433
  0.25463355]]
image (720, 1280, 3)
533
face shape : (127, 127)
kdef [[0.34472886 0.12177137 0.21692178 0.00084008 0.0112845  0.00110153
  0.30335188]]
fer wrong [[1.5232115e-02 5.0167524e-04 2.314

face shape : (125, 125)
kdef [[1.8661421e-02 7.4011159e-01 9.3534729e-03 4.5366606e-04 7.1966566e-02
  1.4322090e-01 1.6232429e-02]]
fer wrong [[2.3144642e-02 7.3000076e-05 2.3736546e-02 7.3287757e-03 7.0323929e-02
  3.1184359e-03 8.7227470e-01]]
fer right [[2.3144642e-02 7.3000076e-05 2.3736546e-02 7.3287757e-03 8.7227470e-01
  7.0323929e-02 3.1184359e-03]]
overall predict [[0.02090303 0.3700923  0.01654501 0.00389122 0.47212064 0.10677242
  0.00967543]]
image (720, 1280, 3)
533
face shape : (126, 126)
kdef [[1.1875009e-02 7.5298631e-01 5.3295721e-03 3.2204285e-04 6.0381785e-02
  1.5785259e-01 1.1252691e-02]]
fer wrong [[2.3264637e-02 7.0754009e-05 2.2463493e-02 7.0684953e-03 6.8849593e-02
  2.9361497e-03 8.7534690e-01]]
fer right [[2.3264637e-02 7.0754009e-05 2.2463493e-02 7.0684953e-03 8.7534690e-01
  6.8849593e-02 2.9361497e-03]]
overall predict [[0.01756982 0.37652853 0.01389653 0.00369527 0.46786433 0.11335109
  0.00709442]]
image (720, 1280, 3)
533
face shape : (126, 126)
kdef [

face shape : (124, 124)
kdef [[6.3758427e-03 4.8814961e-01 8.0475891e-03 1.5868941e-04 8.9119226e-02
  3.9862198e-01 9.5270649e-03]]
fer wrong [[1.8968673e-02 7.0869530e-05 1.4134525e-02 1.6994387e-02 3.9588932e-02
  3.9146086e-03 9.0632802e-01]]
fer right [[1.8968673e-02 7.0869530e-05 1.4134525e-02 1.6994387e-02 9.0632802e-01
  3.9588932e-02 3.9146086e-03]]
overall predict [[0.01267226 0.24411024 0.01109106 0.00857654 0.49772364 0.21910545
  0.00672084]]
image (720, 1280, 3)
533
face shape : (123, 123)
kdef [[8.5943509e-03 4.0384328e-01 7.0847408e-03 2.9488603e-04 1.2887171e-01
  4.4097564e-01 1.0335504e-02]]
fer wrong [[1.7324343e-02 6.1633589e-05 1.3339849e-02 1.9769752e-02 3.7458174e-02
  3.7717794e-03 9.0827447e-01]]
fer right [[1.7324343e-02 6.1633589e-05 1.3339849e-02 1.9769752e-02 9.0827447e-01
  3.7458174e-02 3.7717794e-03]]
overall predict [[0.01295935 0.20195246 0.01021229 0.01003232 0.5185731  0.23921691
  0.00705364]]
image (720, 1280, 3)
533
face shape : (124, 124)
kdef [

face shape : (125, 125)
kdef [[1.0275947e-02 3.7395540e-01 1.1237059e-02 2.8212511e-04 1.0346776e-01
  4.9235767e-01 8.4240697e-03]]
fer wrong [[1.6816558e-02 6.0799721e-05 1.3454166e-02 2.3301832e-02 3.6734156e-02
  3.9255931e-03 9.0570688e-01]]
fer right [[1.6816558e-02 6.0799721e-05 1.3454166e-02 2.3301832e-02 9.0570688e-01
  3.6734156e-02 3.9255931e-03]]
overall predict [[0.01354625 0.1870081  0.01234561 0.01179198 0.5045873  0.26454592
  0.00617483]]
image (720, 1280, 3)
533
face shape : (126, 126)
kdef [[7.9447431e-03 3.2180965e-01 1.3228402e-02 2.5881309e-04 1.0825748e-01
  5.3184426e-01 1.6656568e-02]]
fer wrong [[1.7747808e-02 6.4242420e-05 1.3449780e-02 2.2164822e-02 3.6935564e-02
  4.0043108e-03 9.0563351e-01]]
fer right [[1.7747808e-02 6.4242420e-05 1.3449780e-02 2.2164822e-02 9.0563351e-01
  3.6935564e-02 4.0043108e-03]]
overall predict [[0.01284628 0.16093695 0.01333909 0.01121182 0.5069455  0.2843899
  0.01033044]]
image (720, 1280, 3)
533
face shape : (125, 125)
kdef [[